In [3]:
import os
import torch
from Scripts.Configs.ConfigClass import Config
from lightning.pytorch.callbacks import  ModelCheckpoint
from Scripts.Models.LossFunctions.HeteroLossFunctions import HeteroLoss2
from Scripts.DataManager.GraphConstructor.GraphConstructor import TextGraphType
from Scripts.DataManager.GraphLoader.YelpGraphDataModule import YelpGraphDataModule
from Scripts.Models.LightningModels.LightningModels import HeteroBinaryLightningModel
from Scripts.Models.ModelsManager.ClassifierModelManager import ClassifierModelManager
from Scripts.Models.GraphEmbedding.HeteroDeepGraphEmbedding6 import HeteroDeepGraphEmbedding6

In [4]:
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 256
edge_type_weights = [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.]

graph_type = TextGraphType.SENTIMENT

metadata = (['dep', 'tag', 'word', 'sentence', 'general', 'sentiment'],
 [('dep', 'dep_word', 'word'),
  ('word', 'word_dep', 'dep'),
  ('tag', 'tag_word', 'word'),
  ('word', 'word_tag', 'tag'),
  ('word', 'seq', 'word'),
  ('general', 'general_sentence', 'sentence'),
  ('sentence', 'sentence_general', 'general'),
  ('word', 'word_sentence', 'sentence'),
  ('sentence', 'sentence_word', 'word'),
  ('word', 'word_sentiment', 'sentiment'),
  ('sentiment', 'sentiment_word', 'word')])

In [10]:
graph_embedding = HeteroDeepGraphEmbedding6(300, 1, metadata, 32, dropout=0.2, edge_type_count=11, edge_type_weights=edge_type_weights)
graph_embedding = graph_embedding.to(device)




callbacks = [
ModelCheckpoint(save_top_k=5, mode='max', monitor='val_acc', save_last=True)
]
loss_func = HeteroLoss2(exception_keys=['word'], enc_factor=0.0001)
optimizer = torch.optim.Adam(graph_embedding.parameters(), lr=0.0045, weight_decay=0.0015)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8, 16, 24, 32, 40, 55], gamma=0.5, verbose=True)
lightning_model = HeteroBinaryLightningModel(graph_embedding,
                                    optimizer=optimizer,
                                    loss_func=loss_func,
                                    learning_rate=0.045,
                                    batch_size=batch_size,
                                    user_lr_scheduler=True,
                                    lr_scheduler=lr_scheduler,
                                    min_lr=0.0005
                                    ).to(device)

Adjusting learning rate of group 0 to 4.5000e-03.


In [11]:
max_epochs=0
model_manager = ClassifierModelManager(graph_embedding, lightning_model, log_name='hetero_model_Yelp_2',device=device, num_train_epoch=max_epochs)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
data_manager =  YelpGraphDataModule(config, True, True, test_size=0.05, val_size=0.05, shuffle=False, start_data_load=0 , end_data_load = 5000, device='cpu', batch_size=batch_size, graph_type=graph_type, load_preprocessed_data = True)
data_manager.load_labels()
data_manager.load_graphs()

filename: C:\Users\fardin\Projects\ColorIntelligence\data/GraphData/Yelp\sentiment\graph_var.txt


 Loding Graphs From File : 100%|██████████| 5/5 [01:23<00:00, 16.75s/it]

self.shuffle: False


In [12]:
max_epochs = max_epochs + 5
model_manager.fit(datamodule=data_manager, max_epochs=max_epochs)

You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: logs/hetero_model_Yelp_2


bbbb


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | HeteroDeepGraphEmbedding6 | 205 K 
1 | loss_func | HeteroLoss2               | 0     
2 | train_acc | BinaryAccuracy            | 0     
3 | val_acc   | BinaryAccuracy            | 0     
4 | test_acc  | BinaryAccuracy            | 0     
--------------------------------------------------------
205 K     Trainable params
23        Non-trainable params
205 K     Total params
0.822     Total estimated model params size (MB)
c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-package

Training: |          | 0/? [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 4.5000e-03.


...

Adjusting learning rate of group 0 to 4.5000e-03.


Validation: |          | 0/? [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 4.5000e-03.


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
model_manager.save_plot_csv_logger(loss_names=['train_loss', 'val_loss'], eval_names=['train_acc_epoch', 'val_acc_epoch'], name_prepend=f'test[{0}]')

model_manager.torch_model = model_manager.torch_model.to(device)
model_manager.save_evaluation(data_manager.val_dataloader(), f'test[{0}])',True, True, True, True, True, True, True, multi_class=True)